### Streaming 응답 예제

In [1]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:8080/v1",
    api_key="EMPTY"
)

response = client.chat.completions.create(
    model="mistral-nemo-12b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Always answer in Korean."},
        {"role": "user", "content": "안녕? OpenAI에 대해 알려줄래?"},
    ],
    stream=True
)

for chunk in response:
    print(chunk.choices[0].delta.content, end=" ")


안녕하세요! OpenAI는 인공지능 연구를 위한 비영리 재단입니다. 2015년에 설립되었으며, 목표는 안전하고 유용한 인공지능을 개발하고, 인공지능의 이점을 널리 퍼뜨리는 것입니다. OpenAI는 다양한 인공지능 기술에 대해 연구하고 있으며, 그 결과물로는 ChatGPT와 같은 대화형 AI 모델이 있습니다.</s> 

In [2]:
completion = client.chat.completions.create(
      model="Mistral-Nemo-12b",
      messages=[
        {"role": "system", "content": "You are a helpful assistant. Always answer in Korean."},
        {"role": "user", "content": "OpenAI에 대해 알려줄래?"}
        ],
      stream=False
)

print(completion.choices[0].message.content)

오픈AI는 인공지능과 관련한 연구를 진행하는 미국의 기업입니다. 2015년에 일론 머스크와 샘 알트만 등 몇 명의 창립자가 설립한 이 회사는 인공지능 기술을 통하여 전 세계의 가치 있는 문제를 해결하고자 노력하고 있습니다. 오픈AI는 다양한 프로젝트와 연구를 진행하고 있으며, 그중 몇 가지는 다음과 같습니다.

1. 자연어 처리(NLP) - 오픈AI는 자연어 처리 기술에 중점을 두고 다양한 프로젝트를 진행하고 있습니다. 예를 들어, 대화형 인공지능 어시스턴트인 메타를 개발하였습니다.
2. 로보틱스 - 오픈AI는 로보틱스 기술에도 관심을 갖고 있으며, 특히 유인 조종 없이 이동하고 작업을 수행하는 로봇을 개발하고 있습니다.
3. 컴퓨터 바둑 - 오픈AI는 인공지능을 바둑에 응용하여, 알파고라는 인공지능 바둑 프로그램을 개발하여 세계 챔피언을 이겼습니다.
4. 자율 주행 차량 - 오픈AI는 자율 주행 차량 기술을 연구하고 있으며, 카네기멜론 대학교와 함께 자율 주행 자동차인 카를 개발하였습니다.

오픈AI는 인공지능의 안전성과 윤리적인 활용에 대해서도 중요하게 여기며, 이러한 이슈에 관해 다양한 논의와 제도를 제시하고 있습니다.


In [ ]:
completion.choices[0].message.content

In [ ]:
import requests

url = "http://localhost:8080/v1/chat/completions"
payload = {
    "messages": [{"role": "user", "content": "Hello, how are you?"}],
    "stream": True
}

response = requests.post(url, json=payload, stream=True)

for line in response.iter_lines():
    if line.detail:
        print(line.decode('utf-8'))


In [ ]:
import uvicorn
import argparse
import torch
from threading import Thread
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

In [ ]:
MODEL_PATH="/Users/ckchoi/Desktop/chatbot/Mistral-Nemo-12b"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            trust_remote_code=True
        )

In [ ]:
messages=[
        {"role": "user", "content": "안녕?"}
    ]

In [ ]:
def generate_response(messages):
    inputs = tokenizer.apply_chat_template(
                        messages,
                        return_tensors="pt",
                        return_dict=True,
                        add_generation_prompt=True,
                        truncation=True,
                        max_length=1024).to('mps')
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)
    generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=1024)
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()
    response = ''
    print(f"streamer: {streamer}")
    for text in streamer:
        response += text + ' '
        print(f"text: {text}")
        yield response

In [ ]:
generate_response(messages)

In [ ]:
for chunk in generate_response(messages):
    print(f"chunk: {chunk}")

In [ ]:
inputs = tokenizer.apply_chat_template(
                        messages,
                        return_tensors="pt",
                        return_dict=True,
                        add_generation_prompt=True,
                        truncation=True,
                        max_length=1024).to('mps')
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)
generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=1024)
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()
for text in streamer:
    print(type(text))